In [1]:
from tkinter  import *
import tkinter.font as font
import string
import re
import numpy as np
from unidecode import unidecode
# Reading and Summarizing Text Data
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download("stopwords")
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import scipy.sparse as sp
from scipy import sparse
from collections import defaultdict


[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [ ]:
# create the main window
root = Tk()
root.geometry()  #"900x900"
root.title("Text Classification")

############ Loading the model ###########

# Load the saved model from a file
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

########## load dic ##########
with open('vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)

idf_values = np.load('idf_values.npy')   

############# clean & preprocessing the text   ###########
def punctuation_removal(messy_text):
    clean_list = [char for char in messy_text if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

def numbers_removal(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d',i):
            list_text_new.append(i)
    return ''.join(list_text_new)

#Removing Emails
def remove_Emails(text):
    pattern=r'\S+@\S+'
    text=re.sub(pattern,'',text)
    return text

#Removing URLS
def remove_URLS(text):
    pattern=r'http\S+'
    text=re.sub(pattern,'',text)
    return text
# removing Special Characters & accents

def special_char_removal(text):
    pat = r'^[a-zA-Z0-9]'
    string = re.sub(pat,'',text[0]+text)
    return unidecode(string)

stop_word = set(STOPWORDS)
#print(stop_word , end = '\n\n')
stop_words = set(stopwords.words('english'))
#print(stop_words)

def remove_stopwords_custom(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords from the list of tokens
    tokens_filtere = [word for word in tokens if word.lower() not in stop_words]
    tokens_filtered = [word for word in tokens_filtere if word.lower() not in stop_word]
    # Join the list of tokens back into a single string
    text_filtered = ' '.join(tokens_filtered)
    # Remove additional unwanted characters using a decoding method
    text_filtered = text_filtered.encode('ascii', 'ignore').decode('utf-8')
    return text_filtered


def get_stem(text):
    stemmer = nltk.porter.PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

nlp = spacy.load('en_core_web_sm')
def get_lem(text):
    text = nlp(text)
    lemmatized_sentence = " ".join([word.lemma_ for word in text])
    return lemmatized_sentence

def Tfidf(new_text):
    #cv_new = TfidfVectorizer(vocabulary=vocabulary, norm=None, use_idf=True, smooth_idf=False, sublinear_tf=False)
    #cv_new._tfidf._idf_diag = sparse.sparse.spdiags(idf_values , diags=0, m=len(idf_values), n=len(idf_values))
    # Transform the new text sample into a feature vector using the new TfidfVectorizer instance
    #X_new = cv_new.transform([new_text])
    
    # Convert the input text to a sequence of tokens.
   #tokens = new_text.split(' ')
    #ocabulary = defaultdict(int)
    # For each token in the input text, look up its index in the vocabulary.
    #ndices = [vocabulary[token] for token in tokens]
    # Use the IDF values to weight the importance of each token.
    new_text_vector = [0] * len(vocabulary)
    for word in new_text.split():
        if word in vocabulary:
            new_text_vector[vocabulary[word]] += 1
    
    weighted_tokens = np.multiply(new_text_vector, idf_values).reshape(1,len(vocabulary))
    
    return weighted_tokens
 
def posOrner(result):
    if result == [1]:
        return "Positive"
    elif result==[0]:
        return "Negative"



########## click button ###########
def click ():
    text = text1.get(1.0, "end-1c").lower()
    text = punctuation_removal(text)
    text = numbers_removal(text)
    text = remove_Emails(text)
    text = remove_URLS(text)
    text = special_char_removal(text)
    text = remove_stopwords_custom(text)
    text = get_stem(text)
    text = get_lem(text)
    text = Tfidf(text)
    # Use the loaded model to make predictions
    y_pred = model.predict(text)
    result = posOrner(y_pred)
    label2.config(text = result)
    


############# creat GUI  ##########



# create the main frames
frame = Frame(root, width=400, height=620, bg="#9E9E9E" )
frame.pack(side="left", fill="both")
myFont = font.Font(size=10 ,weight='bold')


label1 = Label(frame ,font =myFont, text="Enter Text", bg="#9E9E9E" ,height= 3, width=20)
label1.pack(pady=5)

text1 = Text(frame , height= 6, font= myFont, width=150)
text1.pack(pady=5 ,padx= 15)

button1 = Button(frame,font =myFont, text="Classifer" ,bg= "#536DFE" , 
                 command= click , height= 4, width=22)
button1.pack(pady=5)

label2 = Label(frame ,font =myFont, text=" ", bg="#9E9E9E" ,height= 3, width=20)
label2.pack(pady=5)


# run the main loop
root.mainloop()
